El objetivo de este documento es producir un archivo .xlsx final en el que analizar qué precios subir y cuáles no.

#### Comandos útiles.

- *Sumar una arriba.* A
- *Sumar una abajo.* B
- *Borrar.* D + D
- *Convertirla en texto.* M

In [1]:
import math 
import re
import pandas as pd
import openpyxl
from openpyxl import Workbook, load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import NamedStyle, Font, PatternFill, Border, Side, Alignment, Color
from openpyxl.worksheet.dimensions import ColumnDimension, DimensionHolder
from openpyxl.worksheet.worksheet import Worksheet

In [2]:
# Cargar funciones de otros archivos.


%run "Funciones para Excel.ipynb"
%run "Funciones útiles.ipynb"

In [3]:
# Ofertas y precios establecidos.

Ofertas = {'Solo x 15 kg.': 10900,
           'Zimpi x 15 kg.': 11800,
           'Leña x 10 kg.': 1900,
           'Pipeta Hectopar 00-04 kg.': 1650,
           'Pipeta Hectopar 05-10 kg.': 1650,
           'Pipeta Hectopar 10-20 kg.': 1650,
           'Pipeta Hectopar 20-40 kg.': 1650,
           'Pipeta Hectopar 40-60 kg.': 1650,
           'Pipeta Hectopar Gato 00-04 kg.': 1650,
           'Pipeta Hectopar Gato 04-08 kg.': 1650,}

### Etapa 0

In [4]:
#-.

Sistema = 'J:/My Drive/Forraje/Exportar.xls'

In [5]:
#-.

#Dataframe del archivo bajado de Nex.

df_Sistema = pd.read_excel(Sistema)

In [6]:
#-.

# Reemplazar caracteres indeseados en Markup %.

df_Sistema['Markup %'] = df_Sistema['Markup %'].str.replace('%', '')
df_Sistema['Markup %'] = df_Sistema['Markup %'].str.replace(',', '.')

In [7]:
#-.

# Columna Markup % a float.

df_Sistema['Markup %'] = df_Sistema['Markup %'].astype(float)

In [8]:
#-.

# Covertimos nombres de columnas a los deseados, sin acentos.

df_Sistema.rename(columns={'Descripción': 'Descripcion'}, inplace=True)
df_Sistema.rename(columns={'Código': 'Codigo'}, inplace=True)
df_Sistema.rename(columns={'Costo Unitário': 'Costo'}, inplace=True)
df_Sistema.rename(columns={'Categoría': 'Categoria'}, inplace=True)
df_Sistema.rename(columns={'Proveedor Principal': 'Proveedor'}, inplace=True)

### Etapa 1

In [9]:
#-.

# Columnas de interés del archivo Sistema.

columnas_Sistema = ['Codigo','Descripcion','Costo','Markup %','Precio','Proveedor','Categoria']

# Eliminar las columnas que no están en la lista de columnas a mantener
columnas_A_Eliminar = [col for col in df_Sistema.columns if col not in columnas_Sistema]
df_Sistema = df_Sistema.drop(columns=columnas_A_Eliminar)

In [10]:
#-. 

#Renombramos columnas.

df_Sistema.rename(columns={'Codigo': 'Codigo Sistema'}, inplace=True)
df_Sistema.rename(columns={'Descripcion': 'Descripcion Sistema'}, inplace=True)
df_Sistema.rename(columns={'Costo': 'Costo Sistema'}, inplace=True)
df_Sistema.rename(columns={'Precio': 'Precio Sistema'}, inplace=True)
df_Sistema.rename(columns={'Markup %': 'Markup % Sistema'}, inplace=True)
df_Sistema.rename(columns={'Proveedor': 'Proveedor Sistema'}, inplace=True)
df_Sistema.rename(columns={'Categoria': 'Categoria Sistema'}, inplace=True)

### Etapa 2

In [11]:
#-.

Proveedores = f'J:/My Drive/Forraje/Calculados/Costos comparados.csv'

In [12]:
#-.

# Crear el DataFrame utilizando esas columnas
df_Proveedores = pd.read_csv(Proveedores)

In [13]:
#-.

# Diccionario con cada cosa como debe ser.
Correspondencia = {
    "Gato Granel": {"Markup %": "48", "Unidad": "KG"},
    "Gato": {"Markup %": "32", "Unidad": "UN"},
    "Perro Adulto Granel": {"Markup %": "48", "Unidad": "KG"},
    "Perro Adulto": {"Markup %": "32", "Unidad": "UN"},
    "Perro Cachorro Granel": {"Markup %": "48", "Unidad": "KG"},
    "Perro Cachorro": {"Markup %": "32", "Unidad": "UN"},
    "Ropa": {"Markup %": "70", "Unidad": "UN"},
    "Mascotas": {"Markup %": "50", "Unidad": "UN"},
    "Limpieza": {"Markup %": "45", "Unidad": "UN"},
    "Veterinaria": {"Markup %": "60", "Unidad": "UN"},
    "Balanceados Granel": {"Markup %": "45", "Unidad": "KG"},
    "Balanceados": {"Markup %": "32", "Unidad": "UN"},
    "Venenos": {"Markup %": "50", "Unidad": "UN"},
    "Liquidos": {"Markup %": "40", "Unidad": "UN"},
    "Pileta": {"Markup %": "50", "Unidad": "UN"},
    "General": {"Markup %": "50", "Unidad": "UN"},
}

In [14]:
#-.

df_Proveedores['Markup %'] = 0
df_Proveedores['Unidad'] = 0

# Utilizar el método map de pandas para aplicar el mapeo a las columnas B y C basado en los valores de la columna A
df_Proveedores['Markup %'] = df_Proveedores['Categoria'].map(lambda x: Correspondencia[x]['Markup %'])
df_Proveedores['Unidad'] = df_Proveedores['Categoria'].map(lambda x: Correspondencia[x]['Unidad'])


In [15]:
#-.

#Renombramos columnas.

df_Proveedores.rename(columns={'Codigo': 'Codigo Proveedores'}, inplace=True)
df_Proveedores.rename(columns={'Descripcion': 'Descripcion Proveedores'}, inplace=True)
df_Proveedores.rename(columns={'Costo': 'Costo Proveedores'}, inplace=True)
df_Proveedores.rename(columns={'Proveedor': 'Proveedor Proveedores'}, inplace=True)
df_Proveedores.rename(columns={'Categoria': 'Categoria Proveedores'}, inplace=True)
df_Proveedores.rename(columns={'Markup %': 'Markup % Proveedores'}, inplace=True)

### Etapa intermedia. Control

In [16]:
#-. 

# Verificación de cantidad de productos en ambas listas.

if len(df_Sistema['Descripcion Sistema']) != len(df_Proveedores['Descripcion Proveedores']):
    exit()
    
for i in range(len(df_Sistema['Descripcion Sistema'])):
    if df_Sistema['Descripcion Sistema'][i] != df_Proveedores['Descripcion Proveedores'][i]:
        print(f"Diferencia encontrada en la posición {i}:")
        print(f"Valor en df_Sistema['Descripcion Sistema'][{i}]: {df_Sistema['Descripcion Sistema'][i]}")
        print(f"Valor en df_Proveedores['Descripcion Proveedores'][{i}]: {df_Proveedores['Descripcion Proveedores'][i]}")
        exit()

### Etapa 3

In [17]:
#-.

# Creamos un df combinando los dos anteriores.
df = pd.concat([df_Sistema, df_Proveedores], axis=1)

In [18]:
#-.

Columnas_a_Borrar = ["Descripcion Sistema","Markup % Sistema", 'Categoria Proveedores',
           "Codigo Proveedores", "Proveedor Proveedores", "Unidad"]

# Borrar columnas.
df = df.drop(columns=Columnas_a_Borrar)

In [19]:
#-.

# Agregamos columnas nuevas.

columnas_Nuevas = ['Costo Calculado', 'Costo Final', 'Costo Relacion',
                   'Precio Calculado', 'Precio Final', 'Precio Relacion']

# Agregar las nuevas columnas al DataFrame con valores vacíos
for i in columnas_Nuevas:
    df = df.assign(**{i: pd.Series([0] * len(df))})

In [20]:
#-.

#Renombramos columnas.

df.rename(columns={'Codigo Sistema': 'Codigo'}, inplace=True)
df.rename(columns={'Descripcion Proveedores': 'Descripcion'}, inplace=True)
df.rename(columns={'Proveedor Sistema': 'Proveedor'}, inplace=True)
df.rename(columns={'Categoria Sistema': 'Categoria'}, inplace=True)
df.rename(columns={'Markup % Proveedores': 'Markup %'}, inplace=True)

In [21]:
#-.

# Convertimos a float las columnas.
df['Costo Sistema'] = df['Costo Sistema'].astype(float)
df['Costo Proveedores'] = df['Costo Proveedores'].astype(str)
df['Costo Proveedores'] = df['Costo Proveedores'].str.replace(',','.')
df['Costo Proveedores'] = df['Costo Proveedores'].astype(float)

In [22]:
#-.

# Columna Costo Final.
df['Costo Final'] = df.apply(lambda row: max(row['Costo Sistema'], row['Costo Proveedores']), axis=1)


In [23]:
#-.

# Si da NaN, se reemplaza por 0.
df['Costo Final'] = df['Costo Final'].fillna(0)

In [24]:
#-.

#Reordenamos las columnas.

Orden_Columnas = ["Codigo", "Descripcion", "Precio Sistema", "Costo Sistema", "Costo Proveedores", "Costo Final",
                  "Markup %", "Categoria", "Proveedor"]

# Ordenar las columnas del DataFrame según la lista de columnas
df = df.reindex(columns=Orden_Columnas)

### Etapa 4

In [25]:
#-.

Granel = 'J:/My Drive/Forraje/Granel.xlsx'

In [26]:
#-.

# Dataframe de Granel.

df_Granel = pd.read_excel(Granel)

In [27]:
#-. 

# Extracción de números.

df_Granel = Extraer_Ultimos_Numeros(df_Granel, 'Bolsa', 'Kilos')


In [28]:
#-.

# Creamos Costo Bolsa y Costo Kilo
df_Granel['Costo Bolsa'], df_Granel['Costo Kilo'] = '', ''

In [29]:
#-.

# Adicionamos los costos de las bolsas del df.
df_Granel = Comparar_Df_y_Copiar(df_Granel, df, 'Bolsa', 'Descripcion', 'Costo Bolsa', 'Costo Final')

In [30]:
#-.

# Reemplazar los valores vacíos con 0
df_Granel = df_Granel.replace('', 0)

# Convertimos las columnas a float.
df_Granel['Costo Bolsa'] = df_Granel['Costo Bolsa'].astype(float)
df_Granel['Kilos'] = df_Granel['Kilos'].astype(float)


C:\Users\tomas\AppData\Local\Temp\ipykernel_11512\768723876.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_Granel = df_Granel.replace('', 0)


In [31]:
#-. 

# Generamos la columna Costo Kilo.

df_Granel['Costo Kilo'] = df_Granel['Costo Bolsa']/df_Granel['Kilos']

In [32]:
#-.

# Adicionamos los costos de los kilos al df.
df = Comparar_Df_y_Copiar(df, df_Granel, 'Descripcion', 'Granel', 'Costo Proveedores', 'Costo Kilo')

### Etapa 5

In [33]:
#-.

Fraccion = 'J:/My Drive/Forraje/Fraccionables.xls'

In [34]:
#-.

# Dataframe de Fraccionables.

df_Fraccion = pd.read_excel(Fraccion)

In [35]:
#-.

# Extracción de números.

df_Fraccion = Extraer_Ultimos_Numeros(df_Fraccion, 'Fraccionable', 'Unidades')

In [36]:
#-.

# Creamos Costo Bolsa y Costo Kilo
df_Fraccion['Costo Paquete'], df_Fraccion['Costo Unidad'] = '', ''

In [37]:
#-.

# Adicionamos los costos de los paquetes del df.
df_Fraccion = Comparar_Df_y_Copiar(df_Fraccion, df, 'Fraccionable', 'Descripcion', 'Costo Paquete', 'Costo Final')

In [38]:
#-.

# Reemplazar los valores vacíos con 0
df_Fraccion = df_Fraccion.replace('', 0)

# Convertimos las columnas a float.
df_Fraccion['Costo Paquete'] = df_Fraccion['Costo Paquete'].astype(float)
df_Fraccion['Unidades'] = df_Fraccion['Unidades'].astype(float)

C:\Users\tomas\AppData\Local\Temp\ipykernel_11512\3055173048.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_Fraccion = df_Fraccion.replace('', 0)


In [39]:
#-.

# Generamos la columna Costo Kilo.

df_Fraccion['Costo Unidad'] = df_Fraccion['Costo Paquete']/df_Fraccion['Unidades']

In [40]:
#-.

# Ivomec.

df_Fraccion['Costo Unidad'].iloc[34] = df_Fraccion['Costo Unidad'].iloc[34] + df_Fraccion['Costo Unidad'].iloc[33]

C:\Users\tomas\AppData\Local\Temp\ipykernel_11512\629439551.py:5: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_Fraccion['Costo Unidad'].iloc[34] = df_Fraccion['Costo Unidad'].iloc[34] + df_Fraccion['Costo Unidad'].iloc[33]
C:\Users\tomas

In [41]:
#-.

# A otra parte.

# Encontrar la fila donde la columna "Descripcion" es igual a "Envase x 250 cc."
Fila_Botellita = df.loc[df['Descripcion'] == 'Envase x 250 cc.']

# Extraer el valor de la columna "Costo Final" de la fila encontrada
Costo_Botellita = Fila_Botellita['Costo Final'].values[0]

# Utiliza el método .iloc para acceder al primer elemento de la columna y asignarle el nuevo valor
df_Fraccion['Costo Unidad'].iloc[0] = df_Fraccion['Costo Unidad'].iloc[0]*25 + Costo_Botellita


C:\Users\tomas\AppData\Local\Temp\ipykernel_11512\3733208705.py:12: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_Fraccion['Costo Unidad'].iloc[0] = df_Fraccion['Costo Unidad'].iloc[0]*25 + Costo_Botellita
C:\Users\tomas\AppData\Local\Tem

In [42]:
#-.

# Bozales.

Costo = df_Fraccion['Costo Paquete'].iloc[35]

df_Fraccion['Costo Unidad'].iloc[35] = Costo*0.06
df_Fraccion['Costo Unidad'].iloc[36] = Costo*0.10
df_Fraccion['Costo Unidad'].iloc[37] = Costo*0.12
df_Fraccion['Costo Unidad'].iloc[38] = Costo*0.14
df_Fraccion['Costo Unidad'].iloc[39] = Costo*0.16
df_Fraccion['Costo Unidad'].iloc[40] = Costo*0.20
df_Fraccion['Costo Unidad'].iloc[41] = Costo*0.22

C:\Users\tomas\AppData\Local\Temp\ipykernel_11512\3029145534.py:7: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_Fraccion['Costo Unidad'].iloc[35] = Costo*0.06
C:\Users\tomas\AppData\Local\Temp\ipykernel_11512\3029145534.py:7: SettingWith

In [43]:
#-.

# Adicionamos los costos por unidad al df.
df = Comparar_Df_y_Copiar(df, df_Fraccion, 'Descripcion', 'Unidad', 'Costo Proveedores', 'Costo Unidad')

### Etapa 6

In [44]:
#-.

# Convertir Markup a float.

df['Markup %'] = df['Markup %'].astype(float)

In [45]:
#-.

# Definir markup diferencial para algunas pipetas.


# Pulmax Perro.
Pulmax = ['Pipeta Pulmax 02-05 kg.', 'Pipeta Pulmax 05-10 kg.', 'Pipeta Pulmax 10-20 kg.', 
          'Pipeta Pulmax 20-40 kg.', 'Pipeta Pulmax 40-60 kg.']
    
Contador = 50
for i in range (0, len(Pulmax)):
    df = Filtrar_y_Rellenar(df, 'Descripcion', Pulmax[i], 'Markup %', Contador)
    Contador = Contador + 5

    
# Pulmax Gato.
Pulmax_Gato = ['Pipeta Pulmax Gato 00-04 kg.', 'Pipeta Pulmax Gato 04-08 kg.']

Contador = 50 
for i in range (0, len(Pulmax_Gato)):    
    df = Filtrar_y_Rellenar(df, 'Descripcion', Pulmax_Gato[i], 'Markup %', Contador)
    Contador = Contador + 5
    
    
# Hectopar.
Hectopar = ['Pipeta Hectopar 00-04 kg.', 'Pipeta Hectopar 05-10 kg.', 'Pipeta Hectopar 10-25 kg.',
            'Pipeta Hectopar 25-40 kg.', 'Pipeta Hectopar 40-60 kg.']
Contador = 50 
for i in range (0, len(Hectopar)):    
    df = Filtrar_y_Rellenar(df, 'Descripcion', Hectopar[i], 'Markup %', Contador)
    Contador = Contador + 5
    
    
# Hectopar Gato.
Hectopar_Gato = ['Pipeta Hectopar Gato 00-04 kg.', 'Pipeta Hectopar Gato 05-08 kg.']
Contador = 50 
for i in range (0, len(Hectopar_Gato)):    
    df = Filtrar_y_Rellenar(df, 'Descripcion', Hectopar_Gato[i], 'Markup %', Contador)
    Contador = Contador + 5

C:\Users\tomas\AppData\Local\Temp\ipykernel_11512\621744001.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[Columna] = Valor


In [46]:
#-.

# Columna Precio Proveedores.

df['Precio Proveedores'] = df['Costo Proveedores']* (1 + (df['Markup %']/100))


In [47]:
#-.

# Llenamos df de 0 si hay NaN.
df = df.fillna(0)

# Columna de Relaciones.

df['Precio Relacion'] = df['Precio Proveedores'] / df['Precio Sistema']
df['Costo Relacion'] = df['Costo Proveedores'] / df['Costo Sistema']


In [48]:
#-.

# Columna Costo Final.
df['Costo Final'] = df.apply(lambda row: max(row['Costo Sistema'], row['Costo Proveedores']), axis=1)

In [49]:
#-.

# Columna Precio Final.
df['Precio Final'] = df.apply(lambda row: max(row['Precio Sistema'], row['Precio Proveedores']), axis=1)

In [50]:
#-.

# Redondeo de Precio Final.
df['Precio Final'] = df['Precio Final'].apply(lambda x: math.ceil(x / 100) * 100)

In [51]:
#-.

# Columna Markup Final.
df['Markup Final'] = (df['Precio Final']/df['Costo Final']) - 1

In [52]:
#-.

# Columna Markup.
df['Markup %'] = df['Markup %']/100

In [53]:
# Arreglamos precios de ofertas.
for i in range(0, len(df)):
    Producto = df['Descripcion'][i]
    for j in range(0, len(Ofertas)):
        if Producto in Ofertas:
            df.at[i, 'Precio Final'] = Ofertas[Producto]

In [54]:
df[df['Descripcion'] == 'Solo x 15 kg.']

,Codigo,Descripcion,Precio Sistema,Costo Sistema,Costo Proveedores,Costo Final,Markup %,Categoria,Proveedor,Precio Proveedores,Precio Relacion,Costo Relacion,Precio Final,Markup Final
1862,1905982,Solo x 15 kg.,10900.0,8232.0,7448.0,8232.0,0.32,Perro Adulto,Mundo Mascotas,9831.36,0.90196,0.904762,10900,0.324101


In [55]:
#-.

# Ordenar columnas.
Orden = ['Codigo', 'Descripcion', 'Precio Final', 'Costo Final', 'Markup Final',
         'Costo Sistema', 'Costo Proveedores', 'Precio Sistema', 'Precio Proveedores', 
         'Precio Relacion','Costo Relacion', 
         'Markup %', 'Proveedor', 'Categoria']

# Reasigna el DataFrame con el nuevo orden de columnas
df = df[Orden]

In [56]:
#.-

# Mostrar dos decimales en la columna Precio Relacion y Costo Relacion.

# Configurar la opción de Pandas para mostrar dos decimales
pd.options.display.float_format = '{:.2f}'.format

# Configurar Pandas para mostrar todas las filas.
pd.set_option('display.max_rows', None)

df.head(10)

,Codigo,Descripcion,Precio Final,Costo Final,Markup Final,Costo Sistema,Costo Proveedores,Precio Sistema,Precio Proveedores,Precio Relacion,Costo Relacion,Markup %,Proveedor,Categoria
0,1905248,9 Lives Gatitos,3500,2314.00,0.51,2314.00,2313.75,3500.00,3424.35,0.98,1.00,0.48,0,Gato Granel
1,1002,9 Lives Gatitos x 8 kg.,24500,18510.00,0.32,18510.00,17493.00,24500.00,23090.76,0.94,0.95,0.32,Benavidez,Gato
2,1905001,9 Lives Pescado,3000,2000.00,0.50,2000.00,1998.00,3000.00,2957.04,0.99,1.00,0.48,0,Gato Granel
3,28391001,9 Lives Pescado x 15 kg.,39600,29970.00,0.32,29970.00,15903.00,39600.00,20991.96,0.53,0.53,0.32,Benavidez,Gato
4,7209,A Otra Parte Concentrado x 600 cc.,15000,9968.00,0.50,9968.00,9968.00,15000.00,14952.00,1.00,1.00,0.50,El Gallo,Mascotas
5,1905237,"A Otra Parte x 0,25 lts.",1000,625.33,0.60,625.00,625.33,1000.00,938.00,0.94,1.00,0.50,0,Mascotas
6,4438,Abrigo Fantasia N° 0,3300,902.00,2.66,820.00,902.00,3300.00,1353.00,0.41,1.10,0.50,Anea,Mascotas
7,4439,Abrigo Fantasia N° 1,3500,1305.00,1.68,1055.00,1305.00,3500.00,1957.50,0.56,1.24,0.50,Anea,Mascotas
8,4440,Abrigo Fantasia N° 2,3700,1344.00,1.75,1222.00,1344.00,3700.00,2016.00,0.54,1.10,0.50,Anea,Mascotas
9,4441,Abrigo Fantasia N° 3,3900,1518.00,1.57,1518.00,1518.00,3900.00,2277.00,0.58,1.00,0.50,Anea,Mascotas


### Etapa 7

In [57]:
#-. 

# Database entero.

# Guardar archivo.
Ruta = 'J:/My Drive/Forraje/Resultados/Resultados.xlsx'

# Guardar
df.to_excel(Ruta, index=False)

In [58]:
#-. 

# Cargamos Excel.
Libro = load_workbook(Ruta)

In [59]:
#-. 

# Seleccionar hoja activa.
Hoja = Libro.active

In [60]:
#-. 

# Anchos.
Hoja.column_dimensions["A"].width = 15 
Hoja.column_dimensions["B"].width = 40
Hoja.column_dimensions["C"].width = 15 
Hoja.column_dimensions["D"].width = 15 
Hoja.column_dimensions["E"].width = 15 
Hoja.column_dimensions["F"].width = 15 
Hoja.column_dimensions["G"].width = 15 
Hoja.column_dimensions["H"].width = 15 
Hoja.column_dimensions["I"].width = 15 
Hoja.column_dimensions["J"].width = 15 
Hoja.column_dimensions["K"].width = 15 
Hoja.column_dimensions["L"].width = 15 
Hoja.column_dimensions["M"].width = 20 
Hoja.column_dimensions["N"].width = 20

In [61]:
#-. 

# Inmovilizar paneles
Hoja.freeze_panes = 'C2'

In [62]:
#.-

# Formato $ con dos decimales para columnas de precios y costos.
Estilo = NamedStyle(name="currency")

# Decimales.
Estilo.number_format = '$#,##0'

# Columnas.
Columnas_con_Peso = ["F", "G", "H", "I"]

for Columna in Columnas_con_Peso:
    for Celda in Hoja[Columna]:
        Celda.style = Estilo

In [63]:
#.-

# Formato general con dos decimales para columnas de relación.
General = NamedStyle(name="general")

# Decimales.
General.number_format = '0.00'

# Columnas.
for Columna in ["J", "K"]:
    for Celda in Hoja[Columna]:
        Celda.style = General

In [64]:
#.-

# Formato general con porcentaje y sin decimales para columna de markup.
General.number_format = '0%'

# Columnas.
for Columna in ["E", "L"]:
    for Celda in Hoja[Columna]:
        Celda.style = General

In [65]:
#.-

# Formato general con cero decimales para columnas de precio final y costo final.
for Columna in ["C", "D"]:
    for Celda in Hoja[Columna]:
        Celda.number_format = '0'

In [66]:
#-.

# Centrar.
Centrar = Alignment(horizontal='center', vertical='center')

# Iteración sobre cada celda de la fila.

for i in Hoja.columns:
    for Celda in i:
        Celda.alignment = Centrar

In [67]:
#-.

# Duplica la hoja 2 veces.
Hoja2_Nombre = "Hoja2"
Hoja2 = Libro.copy_worksheet(Hoja)
Hoja2.title = Hoja2_Nombre

Hoja3_Nombre = "Hoja3"
Hoja3 = Libro.copy_worksheet(Hoja)
Hoja3.title = Hoja3_Nombre

In [68]:
#-. 

# Formato condicional de color rojo en Costo Final para comparación Costo Final - Costo Sistema anterior.
for Fila in Hoja.iter_rows(min_row=2, min_col=4, max_row=Hoja.max_row, max_col=6):  
    for Celda1, Celda2 in zip(Fila, Fila[2:]):
        if abs(Celda1.value - Celda2.value) < 5:
            None
        else:
            Celda1.fill = PatternFill(start_color="FF6666", end_color="FF6666", fill_type="solid")

In [69]:
#-. 

# Formato condicional de color verde en Precio Final para comparación Precio Final - Precio Sistema.
for Fila in Hoja.iter_rows(min_row=2, min_col=3, max_row=Hoja.max_row, max_col=8):  
    for Celda1, Celda2 in zip(Fila, Fila[5:]):
        if abs(Celda1.value - Celda2.value) < 21:
            None
        else:
            Celda1.fill = PatternFill(start_color="89ac76", end_color="89ac76", fill_type="solid")
            Celda1.font = Font(bold=True)

In [70]:
#-. 

# Formato condicional de color azul en Costo Proveedores para comparación Costo Final - Costo Proveedores (cuando este sea bajo).
for Fila in Hoja2.iter_rows(min_row=2, min_col=4, max_row=Hoja2.max_row-1, max_col=7):  
    for Celda1, Celda2 in zip(Fila, Fila[3:]):
        if abs(Celda1.value - Celda2.value) > 21:
            if Celda2.value == 0:
                None
            else:
                Celda2.fill = PatternFill(start_color="B0E0E6", end_color="B0E0E6", fill_type="solid")
                Celda2.font = Font(bold=True)

In [71]:
#-. 

# Formato condicional de color naranja en Costo Final para comparación Costo Final - Costo Proveedores (cuando este sea alto).
for Fila in Hoja3.iter_rows(min_row=2, min_col=4, max_row=Hoja3.max_row-1, max_col=7):  
    for Celda1, Celda2 in zip(Fila, Fila[3:]):
        if Celda1.value - Celda2.value < 0:
            if Celda2.value == 0:
                None
            else:
                Celda1.fill = PatternFill(start_color="FFA500", end_color="FFA500", fill_type="solid")
                Celda1.font = Font(bold=True)

In [72]:
#-. 

# Formato de filtro a la primera fila de la Hoja 1.
Hoja.auto_filter.ref = Hoja.dimensions

# Obtener las filas que tienen formato condicional aplicado.
Filas_Negrita = set()
for Fila in Hoja.iter_rows(min_row=2, max_row=Hoja.max_row, min_col=3, max_col=3):  
    for Celda in Fila:
        if Celda.font == Font(bold=True): 
            Filas_Negrita.add(Fila[0].row)

# Ocultar todas las filas.
for i in range(2, Hoja.max_row + 1):  # Empezar desde la fila 2 ya que la fila 1 contiene los encabezados
    Hoja.row_dimensions[i].hidden = True

# Mostrar solo las filas con formato condicional aplicado.
for Fila in Filas_Negrita:
    Hoja.row_dimensions[Fila].hidden = False


In [73]:
#-. 

# Contar el número de celdas con negrita.
Negrita = len(Filas_Negrita)

# Escribir el número de celdas con negrita en la fila siguiente a la última fila del documento
Fila_Ultima = Hoja.max_row + 1
Celda_Texto = Hoja.cell(row=Fila_Ultima, column=3)
Celda_Texto.value = "Productos:"
Celda_Texto.alignment = Alignment(horizontal="center")
Celda_Resultado = Hoja.cell(row=Fila_Ultima, column=4)
Celda_Resultado.value = Negrita
Celda_Resultado.alignment = Alignment(horizontal="center")

In [74]:
#-. 

# Formato de filtro a la primera fila de la Hoja 2 y 3.
Hoja2.auto_filter.ref = Hoja2.dimensions
Hoja3.auto_filter.ref = Hoja3.dimensions

In [75]:
#-.

# Obtener las filas que tienen formato condicional aplicado en Hoja 2 y 3.
Filas_Negrita_2 = set()
for Fila in Hoja2.iter_rows(min_row=2, max_row=Hoja2.max_row, min_col=7, max_col=7):  
    for Celda in Fila:
        if Celda.font == Font(bold=True): 
            Filas_Negrita_2.add(Fila[0].row)

In [76]:
#-.

# Ocultar todas las filas en Hoja 2.
for i in range(2, Hoja2.max_row + 1):  # Empezar desde la fila 2 ya que la fila 1 contiene los encabezados
    Hoja2.row_dimensions[i].hidden = True

In [77]:
#-.

# Mostrar solo las filas con formato condicional aplicado en Hoja 2.
for Fila in Filas_Negrita_2:
    Hoja2.row_dimensions[Fila].hidden = False

In [78]:
#-.

# Lo mismo para la Hoja 3. 
    
Filas_Negrita_3 = set()
for Fila in Hoja3.iter_rows(min_row=2, max_row=Hoja3.max_row, min_col=7, max_col=7):  
    for Celda in Fila:
        if Celda.font == Font(bold=True): 
            Filas_Negrita_3.add(Fila[0].row)
            
for i in range(2, Hoja3.max_row + 1):  # Empezar desde la fila 2 ya que la fila 1 contiene los encabezados
    Hoja3.row_dimensions[i].hidden = True           
            
for Fila in Filas_Negrita_3:
    Hoja3.row_dimensions[Fila].hidden = False

In [79]:
#-.

# Inmovilizar paneles en Hoja2 y 3.
Hoja2.freeze_panes = 'C2'
Hoja3.freeze_panes = 'C2'

In [80]:
#-.

# Calcula el número de filas en la Hoja2.
Filas = Hoja2.max_row

# Itera sobre todas las filas y mueve los valores de la columna
for i in range(1, Filas):
    # Obtiene el valor de la celda en la columna de origen
    Valor_Celda_Origen = Hoja2.cell(row=i, column=7).value
    
    # Inserta el valor en la nueva posición
    Hoja2.cell(row=i, column=5).value = Valor_Celda_Origen

# Elimina la columna original
Hoja2.delete_cols(7)

In [81]:
#-.

# Calcula el número de filas en la Hoja3.
Filas = Hoja3.max_row

# Itera sobre todas las filas y mueve los valores de la columna
for i in range(1, Filas):
    # Obtiene el valor de la celda en la columna de origen
    Valor_Celda_Origen = Hoja3.cell(row=i, column=7).value
    
    # Inserta el valor en la nueva posición
    Hoja3.cell(row=i, column=5).value = Valor_Celda_Origen

# Elimina la columna original
Hoja3.delete_cols(7)

In [82]:
#.-

# Formato para las columnas de interés en Hoja2.

# Decimales.
General.number_format = '0'

# Columnas.
for Columna in ["D","E"]:
    for Celda in Hoja2[Columna]:
        Celda.style = General
        Celda.alignment = Centrar
        Celda.fill = PatternFill(start_color="808080", end_color="808080", fill_type="solid")
        
for Celda in Hoja2["E"]:
    Celda.fill = PatternFill(start_color="87CEEB", end_color="87CEEB", fill_type="solid")

In [83]:
#.-

# Formato para las columnas de interés en Hoja3.

# Decimales.
General.number_format = '0'

# Columnas.
for Columna in ["D","E"]:
    for Celda in Hoja3[Columna]:
        Celda.style = General
        Celda.alignment = Centrar
        Celda.fill = PatternFill(start_color="808080", end_color="808080", fill_type="solid")
        
for Celda in Hoja3["E"]:
    Celda.fill = PatternFill(start_color="FFA500", end_color="FFA500", fill_type="solid")

In [84]:
#.-

# Cambiar nombres de hojas.
Hoja.title = 'Precio Calculado Alto'
Hoja2.title = 'Costo Calculado Bajo'
Hoja3.title = 'Costo Calculado Alto'

In [85]:
#-. 

# Guardar el archivo
Libro.save(Ruta)

In [86]:
#-.

# Generación de archivo con Costos Proveedor más altos que el Costo Sistema.

# Creamos df.
df_Costo_Alto = df[df['Costo Sistema'] < df['Costo Proveedores']]

In [87]:
#-.

# Generación de archivo con Costos Proveedor más bajos que el Costo Sistema.

# Creamos df.
df_Costo_Bajo = df[df['Costo Sistema'] > df['Costo Proveedores']]

In [88]:
#-.

# Generación de archivo con Precios Proveedor más altos que el Costo Sistema.

# Creamos df.
df_Precio_Alto = df[df['Precio Sistema'] < df['Precio Proveedores']]

In [89]:
#-.

# Generación de archivo con Precios Proveedor más bajos que el Costo Sistema.

# Creamos df.
df_Precio_Bajo = df[df['Precio Sistema'] > df['Precio Proveedores']]

In [90]:
#-.

# Generación de archivo con Costos nulos.

# Creamos df.
df_Costo_Nulo = df[(df['Costo Sistema'] == 0) & (df['Costo Proveedores'] == 0)]

In [91]:
#.-

# Lista con los df.
Lista_Df = [df_Costo_Alto, df_Costo_Bajo, df_Precio_Alto, df_Precio_Bajo, df_Costo_Nulo]

In [92]:
#.-

# Lista con los nombres que vamos a poner en las hojas de cálculo.
Lista_Nombres = ['Costo Alto', 'Costo Bajo', 'Precio Alto', 'Precio Bajo', 'Costo Nulo']

In [93]:
#.-

# Generamos el archivo con el análisis detallado de la generación de precios.
# Ruta_Completa = 'J:/My Drive/Forraje/Resultados/Análisis de precios.xlsx'

# Excel_Con_Df_Por_Hoja(Ruta_Completa, Lista_Df, Lista_Nombres)